In [78]:
from interface import *
import random
random.seed(42)

In [79]:
## Test code for design 18 (MAC chip)
print("{:06b}".format(18))

# instantiate and enable
chip = Chip("/dev/tty.usbmodem2101", 115200)
chip.step_clock(10)

010010


In [80]:
def shift_in(x1, x2):
    x1 = f"{x1:08b}"
    x2 = f"{x2:08b}"
    
    for i in range(8):
        chip.set_input(9, int(x1[i]))  # shiftA
        chip.set_input(10, int(x2[i])) # shiftB
        chip.step_clock(2)
        chip.set_input(8, 1) # shift
        chip.step_clock(2)
        chip.set_input(8, 0) # shift
        chip.step_clock(2)
        
def shift_out():
    bits = []
    for i in range(20):
        bits.append(chip.get_output(9) & 0x1)
        chip.step_clock(2)
        chip.set_input(8, 1) # shift
        chip.step_clock(2)
        chip.set_input(8, 0) # shift
        chip.step_clock(2)
    
    return int("".join(str(x) for x in bits[::-1]), 2)

In [81]:
chip.set_all_inputs(0x000)
chip.reset_sequence()
chip.step_clock(20)
chip.set_input(11, 1) # start
chip.step_clock(10)
chip.set_input(11, 0) # start
chip.step_clock(100)

expected = 0

for i in range(9):
    x1 = random.randint(0, 255)
    x2 = random.randint(0, 255)
    expected += x1*x2
    
    shift_in(x1, x2)
    chip.step_clock(20)
    chip.set_input(7, 1) # do next
    chip.step_clock(3)
    chip.set_input(7, 0) # do next
    chip.step_clock(10)
    
while not chip.get_output(10):
    chip.step_clock()

chip.step_clock(2)
print("result", shift_out())
print("expected", expected)

result 71217
expected 71217
